<a href="https://colab.research.google.com/github/brentwilder/alpine3d-modeling/blob/main/get_prism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download PRISM data from GEE



In [ ]:
# Run all of the things needed to get GEE going in colab
!pip install -q earthengine-api
!earthengine authenticate
!pip install -q geemap
from google.colab import auth
auth.authenticate_user()
import ee
ee.Initialize()
import geemap 

In [18]:
# Define study domain
ulx = -117
uly = 45
lrx = -114
lry = 43
area = ee.Feature(ee.Geometry.Rectangle(ulx,lry,lrx,uly))

In [19]:
# Call data from earth engine
def clip(image):
  return image.clip(area)

collection = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').select('tmean').filterDate('1981-10-01', '2021-10-01').map(clip)

In [20]:
# Download the data
collection.aggregate_array('system:index').getInfo()
geemap.ee_export_image_collection_to_drive(collection, folder='get_prism',crs='EPSG:4326',scale=4000)

Total number of images: 480

Exporting 198110 ...
Exporting 198111 ...
Exporting 198112 ...
Exporting 198201 ...
Exporting 198202 ...
Exporting 198203 ...
Exporting 198204 ...
Exporting 198205 ...
Exporting 198206 ...
Exporting 198207 ...
Exporting 198208 ...
Exporting 198209 ...
Exporting 198210 ...
Exporting 198211 ...
Exporting 198212 ...
Exporting 198301 ...
Exporting 198302 ...
Exporting 198303 ...
Exporting 198304 ...
Exporting 198305 ...
Exporting 198306 ...
Exporting 198307 ...
Exporting 198308 ...
Exporting 198309 ...
Exporting 198310 ...
Exporting 198311 ...
Exporting 198312 ...
Exporting 198401 ...
Exporting 198402 ...
Exporting 198403 ...
Exporting 198404 ...
Exporting 198405 ...
Exporting 198406 ...
Exporting 198407 ...
Exporting 198408 ...
Exporting 198409 ...
Exporting 198410 ...
Exporting 198411 ...
Exporting 198412 ...
Exporting 198501 ...
Exporting 198502 ...
Exporting 198503 ...
Exporting 198504 ...
Exporting 198505 ...
Exporting 198506 ...
Exporting 198507 ...
Expor

In [ ]:
# in case I need to cancel jobs
!earthengine task cancel all

In [ ]:
# FINALLY, use GDAL to convert tif to netcdf
import gdal
